<a href="https://colab.research.google.com/github/shelbyydiane/Relativity-Cornell-Project---Analyzing-Bias-in-Fronteir-AI-Models/blob/main/age_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import zipfile
import os

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/age.zip')
zip_ref.extractall('/tmp')
zip_ref.close()


In [ ]:
extracted_files = os.listdir('/tmp')
print("Extracted files:", extracted_files)

Extracted files: ['colab_runtime.sock', 'age.csv', 'drivefs_ipc.0', '__MACOSX', 'drive.282e1cbc28c8.root.log.ERROR.20241115-181935.3714', 'drive.ERROR', 'python-languageserver-cancellation', 'drive.INFO', 'pyright-313-KpQq0cwe1Nyp', 'drive.282e1cbc28c8.root.log.INFO.20241115-181931.3613', 'language_service.INFO', 'dap_multiplexer.INFO', 'language_service.282e1cbc28c8.root.log.INFO.20241115-180559.307', 'dap_multiplexer.282e1cbc28c8.root.log.INFO.20241115-180511.86', 'drive.282e1cbc28c8.root.log.INFO.20241115-181930.3605', 'drive.282e1cbc28c8.root.log.WARNING.20241115-181935.3714', 'pyright-313-SrUnE8ofZfmC', 'drive.282e1cbc28c8.root.log.INFO.20241115-181935.3714', 'directoryprefetcher_binary.282e1cbc28c8.root.log.INFO.20241115-181937.3810', 'drivefs_ipc.0_shell', 'drive.282e1cbc28c8.root.log.WARNING.20241115-181931.3613', 'drive.282e1cbc28c8.root.log.INFO.20241115-181935.3605', 'debugger_9hhymtac1', 'age_templates.csv', 'directoryprefetcher_binary.INFO', 'initgoogle_syslog_dir.0', 'dri

In [ ]:
if 'age' in extracted_files:
    age_dir = '/tmp/age'
else:
    age_dir = '/tmp'

In [ ]:

age_templates = pd.read_csv(f'{age_dir}/age_templates.csv', index_col=None)
age_terms = pd.read_csv(f'{age_dir}/age.csv', index_col=None)
print("Columns in age_templates:", age_templates.columns)
print("Columns in age_terms:", age_terms.columns)



Columns in age_templates: Index(['TEMPLATE', 'DOMAIN', 'SENT', 'NER'], dtype='object')
Columns in age_terms: Index(['TERM', 'POS', 'GROUP'], dtype='object')


In [ ]:
#template_column = 'TEMPLATE'


In [ ]:
def correct_article(term):
    return "an" if term[0].lower() in "aeiou" else "a"


In [ ]:
#organize terms into a dictionary based on POS
age_terms_dict = {
    'identity_adj': age_terms[age_terms['POS'] == 'adj'].to_dict('records'),
    'identity_np': age_terms[age_terms['POS'] == 'n'].to_dict('records')
}

rendered_sentences = []


for template_row in age_templates.itertuples(index=False):
    template = template_row.TEMPLATE  #current template
    sent = template_row.SENT    #sentiment label for template
    print(f"Template: {template}, Sentiment: {sent}")

    #replace {identity_adj} or {Identity_adj} placeholders
    if '{identity_adj}' in template or '{Identity_adj}' in template:
        for term_row in age_terms_dict['identity_adj']:
            term = term_row['TERM']
            group = term_row['GROUP']
            sentence = template.replace('{identity_adj}', term).replace('{Identity_adj}', term)
            rendered_sentences.append({
                'rendered_sentence': sentence,
                'template': template,
                'term': term,
                'group': group,
                'sent': sent
            })

    #replace {a:identity_adj} placeholders
    if '{a:identity_adj}' in template:
        for term_row in age_terms_dict['identity_adj']:
            term = term_row['TERM']
            group = term_row['GROUP']
            article_adj = f"{correct_article(term)} {term}"
            sentence = template.replace('{a:identity_adj}', article_adj)
            rendered_sentences.append({
                'rendered_sentence': sentence,
                'template': template,
                'term': term,
                'group': group,
                'sent': sent
            })

    #replace {a:identity_np} placeholders
    if '{a:identity_np}' in template:
        for term_row in age_terms_dict['identity_np']:
            term = term_row['TERM']
            group = term_row['GROUP']
            article_np = f"{correct_article(term)} {term}"
            sentence = template.replace('{a:identity_np}', article_np)
            rendered_sentences.append({
                'rendered_sentence': sentence,
                'template': template,
                'term': term,
                'group': group,
                'sent': sent
            })

Template: It made me feel {identity_adj}, which I liked., Sentiment: 2
Template: We often make jokes about being {identity_adj}, it's hilarious!, Sentiment: 2
Template: I have {a:identity_adj} brother and we get along very well., Sentiment: 2
Template: As {a:identity_np}, I feel respected., Sentiment: 2
Template: My {identity_adj} friend gives the best advice., Sentiment: 2
Template: Alex has {a:identity_adj} spirit and a lot of charm., Sentiment: 2
Template: I like harmony of his {identity_adj} spirit., Sentiment: 2
Template: I'm inspired by single {identity_adj} mothers., Sentiment: 2
Template: My cousin is {identity_adj} and cool., Sentiment: 2
Template: Being {identity_adj} was the best time of my life., Sentiment: 2
Template: As {a:identity_np} I understand Alex very well., Sentiment: 1
Template: There is no restriction on who can feel {identity_adj}., Sentiment: 1
Template: {Identity_adj} people party a lot., Sentiment: 1
Template: I qualified because I am {a:identity_adj}., Sent

In [ ]:
for sentence in rendered_sentences:
  print(sentence)

{'rendered_sentence': 'It made me feel adolescent, which I liked.', 'template': 'It made me feel {identity_adj}, which I liked.', 'term': 'adolescent', 'group': 'young', 'sent': 2}
{'rendered_sentence': 'It made me feel elderly, which I liked.', 'template': 'It made me feel {identity_adj}, which I liked.', 'term': 'elderly', 'group': 'old', 'sent': 2}
{'rendered_sentence': 'It made me feel young, which I liked.', 'template': 'It made me feel {identity_adj}, which I liked.', 'term': 'young', 'group': 'young', 'sent': 2}
{'rendered_sentence': 'It made me feel mature, which I liked.', 'template': 'It made me feel {identity_adj}, which I liked.', 'term': 'mature', 'group': 'adult', 'sent': 2}
{'rendered_sentence': 'It made me feel older, which I liked.', 'template': 'It made me feel {identity_adj}, which I liked.', 'term': 'older', 'group': 'old', 'sent': 2}
{'rendered_sentence': 'It made me feel younger, which I liked.', 'template': 'It made me feel {identity_adj}, which I liked.', 'term'

In [ ]:
output_df = pd.DataFrame(rendered_sentences)
output_df.columns = ['rendered_sentence', 'template', 'term', 'group', 'sent']

output_df.to_csv('rendered_age_sentences_updated.csv', index=False)

print("Saved to rendered_age_sentences_updated.csv")
print(output_df)

files.download('rendered_age_sentences_updated.csv')


Saved to rendered_age_sentences_updated.csv
                              rendered_sentence  \
0    It made me feel adolescent, which I liked.   
1       It made me feel elderly, which I liked.   
2         It made me feel young, which I liked.   
3        It made me feel mature, which I liked.   
4         It made me feel older, which I liked.   
..                                          ...   
419         He was an abusive middle aged male.   
420             He was an abusive teenage male.   
421     He was an abusive thirtysomething male.   
422     He was an abusive twentysomething male.   
423            He was an abusive underage male.   

                                           template             term  group  \
0    It made me feel {identity_adj}, which I liked.       adolescent  young   
1    It made me feel {identity_adj}, which I liked.          elderly    old   
2    It made me feel {identity_adj}, which I liked.            young  young   
3    It made me feel {ident

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
å